# Project Overview

The data test results from all Qld Health facilities.

This includes some community centres but not all.

As of  <span style="color:red">**4th June 2020**</span> in Queensland there were:

| | Queensland | Dataset| % Represented| 
|--|--|--|--|
|**Tests**|208 758|103 983|49.8%| 
|**Cases**|1 060|943|88.96%| 




## Overall structure of files:


|Lab_No | Test_Code | Test_Description | Date_Collected |Time_collected |Sex |Age | Category_Code| Facility_Code |Facility_Description |Result |Result_Comment |
|--|-|-|-|-|- |-|-|-|-|-|-|
|Lab number generated for **each sample** taken.<br><br> Multiple tests can be run on the same lab number.| One of for available tests (see below) | Describes the test. | Date sample was collected |Time sample was collected |Patient sex |Patient age |For Medicare billing; not used in present study. |Facility code where the test was ordered from |Full faclity name |Result |Comment on result. These vary panding on stage of pandemic (see below on how this was handled)|



**TEST CODES**

'NCVPCR' = Screening code; if detected a confirmation test will be ordered under the same laboratory number (3rd Gen)

'CRTAQ' = Confirmatory test for NCVPCR; OR a screening test if NCVPCR or CRPCR havent been done (2nd Gen)

'CRPCR' = Screening code; if detected a confirmation test will be ordered under the same laboratory number (2nd Gen)

'CORPCR' = Older test (1st Gen)

'XPNCV' = Screening code; if detected a confirmation test will be ordered under the same laboratory number (4th Gen)


CRPCR and CRTAQ are FSS codes - if detected in any of these tests then it is assumed that virus particules have been detected



******************

## Wrangling Process

Over all aim is to have 1 test result per patient per day.

1. STEP 1: DEAL WITH NULL/MISSING VALUES or WRONG FORMAT VALUES
    - Missing age and sex values categorised as "Not reported".
    - Drop missing date collected.
    - Put ages into age bracket based on ABS.
    
    
2. STEP 2: CONVERT TO DATE TIME TO DATETIME FORMAT
    - This includes adding a "DateTime collected" column based on Date collected and Time collected columns
   

3. STEP 3: DETERMINE WHETHER EACH ROW IS A POISITIVE OR NEGATIVE TEST
    - 'Not Detected' = NO 
    - 'DETECTED' = YES 
    -  nan = depending on Results_comment
        -As definied by clinicians:
            'This test is currently under evaluation and has not been fully validated.' = **NOT DETECTED** = NO
            'F' = **NOT DETECTED** = NO
            'Not Detected' = **NOT DETECTED** = NO
            'This assay is still undergoing development and is not yet fully validated. Results should be interpreted in association with all other information (clinical and laboratory) on the patient. Reports based on this assay are not currently NATA/RCPA endorsed. This test is currently under evaluation and has not been fully validated.' = **NOT DETECTED** = NO
            'Sample referred for confirmatory testing.~This test is currently under evaluation and has not been fully validated.'] = **DETECTED** = YES
            'np.nan' = **NOT DETECTED** = NO
    - 'See Comment' or 'See comment' = Some of the comments indicate that that the virus is not detected whereas other commends indicate that it is. Overall logic will be to:
        - Identify rows where it explicitly says that the virus is NOT detected and change this to "NO"
        - Identify rows where it explicitly says that the virus IS detected and change this to "YES"
        - Identify rows where there is a NaN and change this to "NO"
        - Where the test has been "referred" - will assume that virus HAS BEEN detected and change this to "YES"
        

4. STEP 4: DETERMINE WHETHER EACH LAB_NO IS A POISITIVE OR NEGATIVE TEST
    - There are several Lab_No's that have duplicate values.
    - Overall logic
        - If the lab results are the **SAME** for the same lab_no then drop the duplciate and keep one result only
        - If the lab results are **DIFFERENT** for the same lab_no, then manual validation by clinician/pathology was required do to variation decision making.
  

5. STEP 5: REMOVE DUPLICATE RESULTS ON THE SAME DAY
    - i.e. Different Lab_No on the same day.
    - As per clinician advice:
        - If same results, then keep one copy only
        - If there is a YES and NO result; keep YES
        
        
6. STEP 6: CREATE FINAL DATAFRAME
    - Remove unneeded columns
    - Add new columns:
        - Number of days since first test
        - Number of days since first positive test
        - Clinical status patient based on results
            - Only one positive test required for a POS status
            - Two negative tests >= 24 hours apart for a NEG status
            - Remove multiple final negative status as these are redunat (ie: once negative and they dont switch back to positive but continue having a negative status)
        - Flag days on which there is a status changes
            - 1 == from positive to negative status
            - 2 == from negative *back* to positive
        - Time (days) to negative status 
            - CLINICALLY = clinical negativity is achieved on the *second* of the two negative tests;
            - BIOLOGICALLY = biological negativity is achieved on the *first* of the two negative tests where the second test is considered to be a confirmation of the biological status;
            - These were both counted twice for a **primaary** and **sensitivity** analysis
                - PRIMARY analyis = patients are censored after they *first* achieve negative status
                - SENSITIVITY analysis (\_sensiv) = patients are followed until the end of their data
                    - If a final negative is achieved then the patient is censored after the first first of the two dates (as above)
                    - if a final negative is not achieved, then the patient is not censored
                    - additional sensitivity analysis done on those that achieve a negative status only; where negative status is achieved by either:
                        - Two neg tests: neg status will be counted as being achieved on the first of these two tests; OR
                        - A final negative tests were no additional data is provided
        - Max number of days since and including first positive test
        - Max number of days before first positive test
        - Number of total tests conducted
        - Number of tests since and including first positive
        - Number of times results changed from postive to negative until negative status is achieved for both primary and sensitivity analysis
        - Number of POSITIVE tests since and including first positive
        - Number of NEGATIVE tests since and including first positive
        - Postcode of first test
        - Facility name of first test
        - Remoteness of first test
    - Save final df


# Import Libraries

In [ ]:
import pandas as pd
from datetime import datetime

import re

import numpy as np

import sys


# Import Data

In [ ]:
#get the data file
df = pd.read_csv("Outputs/MatchedDeidentified/MatchedDeidentifiedData.csv")

#get the ses and postcode file
df_ses = pd.read_csv("Outputs/WrangledData/FacilityPostCodeAreaPopulation.csv") 

df_ses = df_ses[["Facility_Description",
                # Address,
                 "PostCode",
                 "Area"
                ]]


#manually validated Lab_nos - i.e. those with multiple results per lab-No

df_manual = pd.read_excel("Outputs/ForValidation/ForClinicianValidation_Validated.xlsx")


In [ ]:
display(df.head())

print("\n********************************************************************************\n")

display(df.tail())


print("\n********************************************************************************\n")


display(df_ses.head())


print("\n********************************************************************************\n")


display(df_manual.head())

# Explore and clean the data

In [ ]:
print(df.shape)

print(len(df["Lab_No"].unique()))

print(len(df["Patient_ID"].unique()))

In [ ]:
df.dtypes

In [ ]:
df.isna().any()

In [ ]:
#look at the unique values

cols = list(df.columns)

cols = ["Test_Code",
        "Category_Code",
       "Facility_Code",
        "Facility_Description",
       "Sex",
       "Result"]

for i in cols:
    
    print(i)
    print(df[i].unique())
    print("\n")

    
len(df["Lab_No"].unique())

In [ ]:
len(df["Facility_Code"].unique())

In [ ]:
#give the index a name
df = df.set_index("ID")

#set Patient_ID as a factor

df["Patient_ID"] = df["Patient_ID"].astype("category")

In [ ]:
df = df.merge(df_ses, on='Facility_Description', how='left')


print(len(df["Patient_ID"].unique()))
df.head()

In [ ]:
df.dtypes

In [ ]:
list(df.columns)

**********************************

# <span style="color:magenta"> **STEP 1: DEAL WITH NULL or WRONG FORMAT VALUES**<span>

In [ ]:
#check which columns are null
df.isnull().any()

**********************

### <span style="color:magenta">DEALING WITH DATE COLLECTED</span>

In [ ]:
#Find rows that have a date

pattern1 = r"^(19|20)\d\d[- /.](0[1-9]|1[012])[- /.](0[1-9]|[12][0-9]|3[01])$"

detected = df[~df.Date_Collected.str.contains(pattern1)]

print("Number of patients :", len(df["Patient_ID"].unique()))

print(list(detected.Lab_No))

print(detected.shape)

detected.head()



In [ ]:
if len(detected) > 10:
    
    print("TOO MANY MISSING VALUES removed!!!!")
    sys.exit()
    
else:
    #drop these from df
    df = df.drop(detected.index.values)
    
    print(len(detected), " missing dates removed")



len(df.Lab_No.unique())

**********************

### <span style="color:magenta">DEALING WITH AGE</span>


In [ ]:
#put a dummy number in so we can convert to int for imputation
df["Age"] = df["Age"].fillna(-5)

#convert to int
df["Age"] = df["Age"].astype(int)

print(len(df["Lab_No"].unique()))

In [ ]:
#put ages into bins < 65 , and 65 and over

#create bins
bins = np.arange(-5, 86, 5).tolist()
#bins.append(120)

bins = [-5,
        0, 
        17,
        65, 
        200]


#create labels
labels=  ["Not reported", "16 and under", "17 to 64", "65 and over"]

print(labels)

print(bins)

df['AgeGroup'] = pd.cut(df['Age'], bins=bins, labels=labels, right=False)


#df[df["Age"]== -5 ][["Age", "AgeGroup"]]

In [ ]:
print(len(df[df["AgeGroup"] == "Not reported"]), " missing age categorised as 'not reported'")

In [ ]:
print("Number of patients :", len(df["Patient_ID"].unique()))
df.head()

**********************

### <span style="color:magenta">DEALING WITH SEX</span>

In [ ]:
#put a value in for missing sex
df["Sex"] = df["Sex"].fillna("Not reported")


print(len(df[df["Sex"] == "Not reported"]), ' missing sex values coded as "not reported"')



# <span style="color:magenta">**STEP 2: Deal with Date and Time Collected** <span>

In [ ]:
df.dtypes

In [ ]:
#add 0 at front of time

df["Time_collected"] = df["Time_collected"].str.lstrip()

df["Time_collected"] = df['Time_collected'].apply(lambda x: x.zfill(8))

df.head()

In [ ]:
#create a datetime column
df['DateTime_Collected'] = df['Date_Collected'].str.cat(df['Time_collected'],sep=" ")

df.head()

In [ ]:
#convert date columns to datetime
df["DateTime_Collected"] = pd.to_datetime(df["DateTime_Collected"], format='%Y-%m-%d %H:%M:%S', errors='coerce')

df["Date_Collected"] = pd.to_datetime(df["Date_Collected"], format='%Y-%m-%d', errors='coerce')


df.head()

In [ ]:
#first date of data
print("FIRST DATE OF DATA:")
print(df["DateTime_Collected"].min())

print("\n")


print("LAST DATE OF DATA:")
print(df["DateTime_Collected"].max())


print("\n")


print("DAYS OF DATA")
print("Number of days with data: ", len(df["Date_Collected"].unique()))

print("Number of days between first and last test in dataset: ", (df["DateTime_Collected"].max() - df["DateTime_Collected"].min()).days)


print("Number of patients :", len(df["Patient_ID"].unique()))



# <span style="color:magenta"> **STEP 3: CHANGE EACH _ROW_ TO BE DETECTED: YES or NO** <span>
Change codes for detected/not detected to yes/no

In [ ]:
#create column called RESTUL_1.

#If result = DETECTED then RESULT_1 is YES, otherwise put NAN
df['Result_1'] = np.where(df['Result'] == 'DETECTED', "YES", np.nan)

#If result = Not Detected then RESULT_1 is NO otherwise do nothing.
df['Result_1'] = np.where(df['Result'] == 'Not Detected', "NO", df['Result_1'])

df['Result_1'] = np.where(df['Result'] == 'ND', "NO", df['Result_1'])


len(df["Lab_No"].unique())

****************
## <span style="color:magenta">DEALING with NULLS</span>




In [ ]:
#look at rows with nan rest result

df1 = df[df['Result'].isnull()]

print(df1.shape)

print("\n")

print(df1["Result_Comment"].unique())

print("\n")

print(df1["Facility_Description"].unique())

print("\n")


len(df["Lab_No"].unique())

In [ ]:
#create list of unique values that the code already deals with to check if there are new comments that requrie checking

null_comments_deltWith = list(['This test is currently under evaluation and has not been fully validated.', 
                              'F',
                               np.nan,
                              'This assay is still undergoing development and is not yet fully~validated. Results should be interpreted in association with all other~information (clinical and laboratory) on the patient. Reports based on~this assay are not currently NATA/RCPA endorsed.~This test is currently under evaluation and has not been fully validated.',
                              'Sample referred for confirmatory testing.~This test is currently under evaluation and has not been fully validated.',
                              'Not Detected'])



#STOP code if new comments have appeared
if len(set(df1["Result_Comment"].unique()) - set(null_comments_deltWith)) != 0:
            print("ERROR: NEW NULL Comments!\n\n",
                  (set(df1["Result_Comment"].unique()) - set(null_comments_deltWith)))
            sys.exit()

len(df["Lab_No"].unique())

In [ ]:
#replace nulls with ""
df1 = df1.replace(np.nan,"")


#make results upper

df1['Result_Comment'] = df1['Result_Comment'].str.upper()
df1['Result'] = df1['Result'].str.upper()


len(df["Lab_No"].unique())

In [ ]:
patient = list(df1[df1['Result_Comment'] == ""]["Patient_ID"])



for i in patient:
    print(i, "had", len(df1[df1['Patient_ID'] == i]), "tests")
    
    #display(df1[df1['Patient_ID'] == i])
    

In [ ]:
print(len(df["Lab_No"].unique()))

In [ ]:
#DETECTED

pattern1 = r"REFERRED"

detected = df1[df1.Result_Comment.str.contains(pattern1)]

print(list(detected.Lab_No))

display(detected.shape)

detected.head()

#replace values with YES in df
for rows in detected.index:
    df.loc[df.index == rows, "Result_1"] = "YES"   
    
#drop these from df1

df1 = df1[~df1.isin(detected)].dropna()


#see whats left
display(df1["Result_Comment"].unique())

print(len(df["Lab_No"].unique()))
print("Number of patients :", len(df["Patient_ID"].unique()))

In [ ]:
#remaining are not detected
for rows in df1.index:
    df.loc[df.index == rows, "Result_1"] = "NO"   
    
#drop these from df1


df1 = df1[~df1.isin(df1)].dropna()

#see whats left
display(df1["Result_Comment"].unique())

print(len(df["Lab_No"].unique()))
print("Number of patients :", len(df["Patient_ID"].unique()))

****************

## <span style="color:magenta">DEALING with SEE COMMENTS</span>.



In [ ]:
df1 = df[(df["Result"]=='See Comment') | (df["Result"]== 'See comment')]

print(df1["Result_Comment"].unique())

len(df["Lab_No"].unique())

In [ ]:
#replace Nan with ""
df1 = df1.replace(np.nan,"")

df1['Result_Comment'] = df1['Result_Comment'].str.upper()


len(df["Lab_No"].unique())


In [ ]:
#NULL values

nulls = df1[df1.Result_Comment == ""]

nulls.shape


display(nulls.head())


#replace values with NO in df
for rows in nulls.index:
    df.loc[df.index == rows, "Result_1"] = "NO"   
    
#drop these from df1
df1 = df1[~df1.isin(nulls)].dropna()

#see whats left
print(df1["Result_Comment"].unique())


print("Number of patients :", len(df["Patient_ID"].unique()))

len(df["Lab_No"].unique())

In [ ]:
#NOT DETECTED
#if the phrase 'not detected' is in the comments, change results_1 to NO

pattern1 = r"NOT\sDETECTED"

not_detected = df1[df1.Result_Comment.str.contains(pattern1)]

display(not_detected.shape)



#replace values with NO in df
for rows in not_detected.index:
    df.loc[df.index == rows, "Result_1"] = "NO"   
    
#drop these from df1

df1 = df1[~df1.isin(not_detected)].dropna()

#see whats left
print(df1["Result_Comment"].unique())


print("Number of patients :", len(df["Patient_ID"].unique()))
len(df["Lab_No"].unique())

In [ ]:
#DETECTED
#if the phrase 'detected' is in the comments, change results_1 to YES

pattern2 = r"DETECTED"

detected = df1[df1.Result_Comment.str.contains(pattern2)]

detected.shape

detected.head()


#replace values with NO in df
for rows in detected.index:
    df.loc[df.index == rows, "Result_1"] = "YES"   
    
#drop these from df1

df1 = df1[~df1.isin(detected)].dropna()

#see whats left
print(df1["Result_Comment"].unique())

len(df["Lab_No"].unique())

In [ ]:
#Drop those that have been "combined" with another test

pattern3 = r"COMBINED\sWITH\sLAB"

detected = df1[df1.Result_Comment.str.contains(pattern3)]

print(detected.shape)

print("Number of patients :", len(df["Patient_ID"].unique()))

print(len(df["Lab_No"].unique()))

#detected.head()

In [ ]:
#drop these from df
df = df.drop(detected.index.values)

#drop these from df1
df1 = df1[~df1.isin(detected)].dropna()

#see whats left
print(df1["Result_Comment"].unique())

print("Number of patients :", len(df["Patient_ID"].unique()))

len(df["Lab_No"].unique())

In [ ]:
#'This test has been performed on lab #...'
#drop this weird value as it is a duplicate

pattern4 = r'^THIS\sTEST\sHAS\sBEEN\sPERFORMED\sON\sLAB'


detected = df1[df1.Result_Comment.str.contains(pattern4)]

#display(detected)

#drop these from df
df = df.drop(detected.index.values)

#drop these from df1
df1 = df1[~df1.isin(detected)].dropna()

#see whats left
print(df1["Result_Comment"].unique())

len(df["Lab_No"].unique())

In [ ]:
#'UNABLE TO PERFORM NAA TEST #...'
#drop this weird value as it a bung test

pattern4 = r'^UNABLE\sTO\sPERFORM\sNAA\sTEST'


detected = df1[df1.Result_Comment.str.contains(pattern4)]

display(detected)

#drop these from df
df = df.drop(detected.index.values)

#drop these from df1
df1 = df1[~df1.isin(detected)].dropna()

#see whats left
print(df1["Result_Comment"].unique())


print("Number of patients :", len(df["Patient_ID"].unique()))

len(df["Lab_No"].unique())

In [ ]:
#tests that have been referred are considered to have detected virus
pattern2 = r"REFERRED"

detected = df1[df1.Result_Comment.str.contains(pattern2)]

detected.shape

detected.head()

#replace values with YES in df
for rows in detected.index:
    df.loc[df.index == rows, "Result_1"] = "YES"   
    
#drop these from df1

df1 = df1[~df1.isin(detected)].dropna()

#see whats left
print(df1["Result_Comment"].unique())

len(df["Lab_No"].unique())

In [ ]:
#tests that have been referred are considered to have detected virus
pattern2 = r"PROVISIONALLY\sPOSITIVE"

detected = df1[df1.Result_Comment.str.contains(pattern2)]

detected.shape

#display(detected.head())

#replace values with YES in df
for rows in detected.index:
    df.loc[df.index == rows, "Result_1"] = "YES"   
    
#drop these from df1

df1 = df1[~df1.isin(detected)].dropna()

#see whats left
print(df1["Result_Comment"].unique())

print(len(df["Lab_No"].unique()))

df1

In [ ]:
patient = list(df1["Patient_ID"])


for i in patient:
    print(i, "had", len(df1[df1['Patient_ID'] == i]), "tests")

In [ ]:
#remaining are not detected
for rows in df1.index:
    df.loc[df.index == rows, "Result_1"] = "NO"   
    
#drop these from df1


df1 = df1[~df1.isin(df1)].dropna()

#see whats left
display(df1["Result_Comment"].unique())

len(df["Lab_No"].unique())

In [ ]:
#STOP code if new comments have appeared
if len(df1["Result_Comment"].unique()) != 0:
            print("ERROR: NEW NULL Comments!\n\n",
                  df1["Result_Comment"].unique())
            sys.exit()


**************

# <span style="color:magenta"> **STEP 4: REMOVE DUPLICATE LAB_NOs** <span>

Same Lab_No but multiple test results.


In [ ]:
#create a df of duplicate Lab_No

duplicates = df[df.duplicated(['Lab_No'], 
                              keep = False)].sort_values(['Lab_No'],
                                                        ascending= True,
                                                        na_position='first')

#replace Nan with ""
duplicates = duplicates.replace(np.nan, '')

df = df.replace(np.nan, '')

len(df["Lab_No"].unique())

In [ ]:
#Stop code if there are more than 3 results per Lab_No
if duplicates.Lab_No.value_counts().max() > 4:
    print("ERROR: More than 4 results per Lab_No")
    sys.exit()


****************

## <span style="color:magenta">TWO DUPLICATES </span>



In [ ]:
duplicates.columns

In [ ]:
#create a df with duplicates that only have two results per lab_no

two_duplicates = duplicates[duplicates.groupby("Lab_No")["Lab_No"].transform('size') == 2][['Lab_No',
                                                                                            "Test_Code",
                                                                                            "Result",
                                                                                            "Result_Comment",
                                                                                            'Result_1']]
print(two_duplicates.Test_Code.unique())

print(len(two_duplicates))

two_duplicates.head()

In [ ]:
#drop duplicates that have the same result
two_duplicates = two_duplicates.drop_duplicates(['Lab_No',
                                                 'Result_1'])

two_duplicates = two_duplicates.sort_values(by=['Lab_No',
                                               "Result_1"])


print(len(two_duplicates))

two_duplicates.head(15)

In [ ]:
#get IDs of those who dont have duplicate values - these are the ones that have only 2 duplicates and the same result for both test
IDs = two_duplicates[~two_duplicates.duplicated(['Lab_No'], keep = False)].sort_values(['Lab_No'],
                                                                                       ascending= True,
                                                                                       na_position='first')
print("IDs", IDs.shape)
print("df before", df.shape)

#drop these from df
df = df[~df.isin(IDs)].dropna()

print("df after", df.shape)

len(df["Lab_No"].unique())

In [ ]:
#get those with duplicate lab_no remaining - these are those with DIFFERENT lab results
two_duplicates = two_duplicates[two_duplicates.duplicated(['Lab_No'], keep = False)].sort_values(['Lab_No'],
                                                        ascending= True,
                                                        na_position='first')

print(len(two_duplicates))

two_duplicates.head(6)

In [ ]:
#tests the code can deal with
tests = ['CRTAQ', 'NCVPCR', 'CRPCR']



#STOP code if new test codes have appeared
if len(set(two_duplicates.Test_Code.unique()) - set(tests) ) != 0:
    
            print("ERROR: NEW TEST CODES!\n\n",
                  set(two_duplicates.Test_Code.unique()) - set(tests))
        
            sys.exit()

In [ ]:
nulls = two_duplicates.pivot(index='Lab_No', columns='Test_Code', values=['Result_1',
                                                                           "Result",
                                                                           'Result_Comment'])

#throw an error if there are any nulls in the CRTAQ column

if len(nulls[nulls["Result_1"]["CRTAQ"].isnull()]) > 0:
    print("ERROR: Nulls in CRTAQ")
    sys.exit()

    
    
two_duplicates.pivot(index='Lab_No', columns='Test_Code', values=['Result_1',
                                                                  "Result",
                                                                  'Result_Comment'])

In [ ]:
#create a list of IDs to KEEP
#IDs_keep

#where either CRPCR is YES or CRTAQ is YES

IDs = two_duplicates[(two_duplicates["Test_Code"] == "CRTAQ") | (two_duplicates["Test_Code"] == "CRPCR")]

print(len(IDs.Lab_No.unique()))

#keep the yes values only if duplicate Lab_IDs

#drop duplicates that have the same result
IDs = IDs.drop_duplicates(['Lab_No',
                           'Result_1'])

#get the duplicates for CRPCR and CRTAQ with a different result
IDs_duplicate = IDs[IDs.duplicated(['Lab_No'], 
                              keep = False)].sort_values(['Lab_No'],
                                                        ascending= True,
                                                        na_position='first')

#get the ones that say no
IDs_duplicate = IDs_duplicate[IDs_duplicate.Result_1 == "NO"]

#remove these from IDs
IDs = IDs[~IDs.isin(IDs_duplicate)].dropna()


print(len(IDs.Lab_No.unique()))


#save csv for validation
IDs[["Lab_No",
    "Result_1"]].to_csv("Outputs/ForValidation/TwoLabNumbers.csv")

########################################

#create df of IDs to REMOVE - drop two duplicates
print(len(two_duplicates.Lab_No.unique()))

IDs = two_duplicates[~two_duplicates.isin(IDs)].dropna()

print(len(IDs.Lab_No.unique()))


##############################################

#remove from df

print("IDs", IDs.shape)

print("df before", df.shape)

#drop these from df
df = df[~df.isin(IDs)].dropna()

print("df after", df.shape)

len(df["Lab_No"].unique())



**********************

### <span style="color:magenta">THREE DUPLICATES</span>


In [ ]:
#create a df with duplicates that only have three results per lab_no

thee_duplicates = duplicates[duplicates.groupby("Lab_No")["Lab_No"].transform('size') == 3][['Lab_No',
                                                            "Test_Code",
                                                            "Result",
                                                            "Result_Comment",
                                                            'Result_1']]

print(len(thee_duplicates))
display(thee_duplicates.head())


display(thee_duplicates.pivot(index='Lab_No', columns='Test_Code', values=['Result_1',
                                                                           "Result",
                                                                           'Result_Comment']).head(15))




#tests the code can deal with
tests = ['CRTAQ', 'NCVPCR', 'CRPCR']




            
            
nulls = thee_duplicates.pivot(index='Lab_No', columns='Test_Code', values=['Result_1',
                                                                           "Result",
                                                                           'Result_Comment'])

#throw an error if there are any nulls in the CRTAQ column

if len(nulls[nulls["Result_1"]["CRTAQ"].isnull()]) > 0:
    print("ERROR: Nulls in CRTAQ")
    sys.exit()
   

In [ ]:
#create a list of IDs to KEEP
#IDs_keep

#where either CRPCR is YES or CRTAQ is YES

IDs = thee_duplicates[(thee_duplicates["Test_Code"] == "CRTAQ") | (thee_duplicates["Test_Code"] == "CRPCR")]

print(len(IDs.Lab_No.unique()))


#drop duplicates that have the same result
IDs = IDs.drop_duplicates(['Lab_No',
                           'Result_1'])


#get the duplicates for CRPCR and CRTAQ with a different result
IDs_duplicate = IDs[IDs.duplicated(['Lab_No'], 
                              keep = False)].sort_values(['Lab_No'],
                                                        ascending= True,
                                                        na_position='first')

#get the ones that say no
IDs_duplicate = IDs_duplicate[IDs_duplicate.Result_1 == "NO"]

#remove these from IDs
IDs = IDs[~IDs.isin(IDs_duplicate)].dropna()


print(len(IDs.Lab_No.unique()))

display(IDs[["Lab_No",
    "Result_1"]])

#save csv for validation
IDs[["Lab_No",
    "Result_1"]].to_csv("Outputs/ForValidation/ThreeLabNumbers.csv")

########################################

#create df of IDs to REMOVE - drop two duplicates
print(len(thee_duplicates.Lab_No.unique()))

IDs = thee_duplicates[~thee_duplicates.isin(IDs)].dropna()

print(len(IDs.Lab_No.unique()))


##############################################

#remove from df

print("IDs", IDs.shape)

print("df before", df.shape)

#drop these from df
df = df[~df.isin(IDs)].dropna()

print("df after", df.shape)

len(df["Lab_No"].unique())




**********************


### <span style="color:magenta">FOUR DUPLICATES</span>

In [ ]:
#create a df with duplicates that only have three results per lab_no

four_duplicates = duplicates[duplicates.groupby("Lab_No")["Lab_No"].transform('size') == 4][['Lab_No',
                                                            "Test_Code",
                                                            "Result",
                                                            "Result_Comment",
                                                            'Result_1']]

print(len(four_duplicates))
display(four_duplicates.head())


display(four_duplicates.pivot(index='Lab_No', columns='Test_Code', values=['Result_1',
                                                                           "Result",
                                                                           'Result_Comment']))





In [ ]:
nulls = four_duplicates.pivot(index='Lab_No', columns='Test_Code', values=['Result_1',
                                                                           "Result",
                                                                           'Result_Comment'])

#throw an error if there are any nulls in the CRTAQ column

if len(nulls[nulls["Result_1"]["CRTAQ"].isnull()]) > 0:
    print("ERROR: Nulls in CRTAQ")
    sys.exit()
   

In [ ]:
#create a list of IDs to KEEP
#IDs_keep

#where either CRPCR is YES or CRTAQ is YES

IDs = four_duplicates[(four_duplicates["Test_Code"] == "CRTAQ") | (four_duplicates["Test_Code"] == "CRPCR")]

print(len(IDs.Lab_No.unique()))


#drop duplicates that have the same result
IDs = IDs.drop_duplicates(['Lab_No',
                           'Result_1'])


#get the duplicates for CRPCR and CRTAQ with a different result
IDs_duplicate = IDs[IDs.duplicated(['Lab_No'], 
                              keep = False)].sort_values(['Lab_No'],
                                                        ascending= True,
                                                        na_position='first')

#get the ones that say no
IDs_duplicate = IDs_duplicate[IDs_duplicate.Result_1 == "NO"]

#remove these from IDs
IDs = IDs[~IDs.isin(IDs_duplicate)].dropna()


print(len(IDs.Lab_No.unique()))


#save csv for validation
IDs[["Lab_No",
    "Result_1"]].to_csv("Outputs/ForValidation/FourLabNumbers.csv")

########################################

#create df of IDs to REMOVE - drop two duplicates
print(len(four_duplicates.Lab_No.unique()))

IDs = four_duplicates[~four_duplicates.isin(IDs)].dropna()

print(len(IDs.Lab_No.unique()))


##############################################

#remove from df

print("IDs", IDs.shape)

print("df before", df.shape)

#drop these from df
df = df[~df.isin(IDs)].dropna()

print("df after", df.shape)

len(df["Lab_No"].unique())



In [ ]:
#check that duplicates have been removed
first_dedup_length = len(df)

print("Number of rows: ", first_dedup_length)

print("\n")

print("Number of tests: ", len(df.Lab_No.unique()))

print("\n")

if len(df) != len(df.Lab_No.unique()):
    
    print("Duplicates not removed!!!!")
    sys.exit()
    
else:
    print("All duplicates removed")
                   
                                    

### Update Lab_No with manually varified results
Lab_Nos are manually varified by clinician

In [ ]:
#Replace Lab_No result with manually varified result

#If result = DETECTED then RESULT_1 is YES, otherwise put NO
df_manual['Result_1'] = np.where(df_manual['ClinicianDeterminedResult'] == 'D', "YES", "NO")

#reset the index to ID
df_manual = df_manual.set_index('ID')

df_manual.head()

In [ ]:
#name the index of df
df.index.name = 'ID'

df.head()

In [ ]:
#replace values
df.loc[df.Lab_No.isin(df_manual.Lab_No), ['Result_1']] = df_manual[['Result_1']]

In [ ]:
#save lab tests
df[['Patient_ID',
    'Lab_No', 
    'Date_Collected',
    'Time_collected',
    'Sex',
    'Age',
    "Result_1",
    'DateTime_Collected',
    'Area']].to_csv("Outputs/WrangledData/tests_area.csv") 

# <span style="color:magenta"> **STEP 5: REMOVE DUPLUCATE RESULTS ON THE SAME DAY** </span>



In [ ]:
df.columns

In [ ]:
df = df.sort_values(['DateTime_Collected'],
               ascending= True,
               na_position='first')

In [ ]:
#Number of tests taken at the same time on the same day with the same result

removed = len(df[df.duplicated([#'Lab_No', 
                  #'Test_Code', 
                  #'Test_Description',
                  'Date_Collected',
                  #'Time_collected',
                  #'UrNo', 
                  #'Surname', 
                  #'First_Names', 
                  #'Date_of_Birth',
                  #'Sex', 
                  #'Age', 
                  #'Category_Code', 
                  #'Facility_Code', 
                  #'Facility_Description',
                  #'Result', 
                  #'Result_Comment', 
                  #'DateTime_Collected',
                  'Result_1',
    'Patient_ID'])])
              
removed

In [ ]:
#drop duplicates that have the same result for different tests on the same time/day

df = df.drop_duplicates([#'Lab_No', 
                  #'Test_Code', 
                  #'Test_Description',
                  'Date_Collected',
                  #'Time_collected',
                  #'UrNo', 
                  #'Surname', 
                  #'First_Names', 
                  #'Date_of_Birth',
                  #'Sex', 
                  #'Age', 
                  #'Category_Code', 
                  #'Facility_Code', 
                  #'Facility_Description',
                  #'Result', 
                  #'Result_Comment', 
                  #'DateTime_Collected',
                  'Result_1',
                  'Patient_ID'])

df = df.sort_values(by=['Lab_No',
                        "Result_1"])


print(len(df[df.duplicated([#'Lab_No', 
                  #'Test_Code', 
                  #'Test_Description',
                  'Date_Collected',
                  #'Time_collected',
                  #'UrNo', 
                  #'Surname', 
                  #'First_Names', 
                  #'Date_of_Birth',
                  #'Sex', 
                  #'Age', 
                  #'Category_Code', 
                  #'Facility_Code', 
                  #'Facility_Description',
                  #'Result', 
                  #'Result_Comment', 
                  #'DateTime_Collected',
                  'Result_1',
                  'Patient_ID'], keep = False)].sort_values(['Patient_ID'],
                                                        ascending= True,
                                                        na_position='first')))

#df.head()

In [ ]:
#check that duplicates have been removed
second_dedup_length = len(df)

print("Number of rows: ", second_dedup_length)

print("\n")

print("Should be: ", first_dedup_length - removed)

print("\n")

if first_dedup_length - removed != second_dedup_length:
    
    print("Duplicates not removed!!!!")
    sys.exit()
    
else:
    print("All duplicates removed")
                   

print("\n")

print("Unique patients: ", len(df.Patient_ID.unique()))

print("\n")

print("Unique tests: ", len(df))

In [ ]:
#Need to keep "YES" for days with YES and NO results on the same day

#get rows that more than one test for the patien on the same day
duplicates = df[df.duplicated([#'Lab_No', 
                  #'Test_Code', 
                  #'Test_Description',
                  'Date_Collected',
                  #'Time_collected',
                  #'UrNo', 
                  #'Surname', 
                  #'First_Names', 
                  #'Date_of_Birth',
                  #'Sex', 
                  #'Age', 
                  #'Category_Code', 
                  #'Facility_Code', 
                  #'Facility_Description',
                  #'Result', 
                  #'Result_Comment', 
                  #'DateTime_Collected',
                  #'Result_1',
                  'Patient_ID'], keep = False)].sort_values(['Patient_ID'],
                                                        ascending= True,
                                                        na_position='first')


display(duplicates.head())

duplicates.shape

In [ ]:
#get values where Result_1 is NO
IDs = duplicates[duplicates["Result_1"]== "NO"]

print("IDs", IDs.shape)

print("df before", df.shape)

#drop these from df
df = df[~df.isin(IDs)].dropna()

print("df after", df.shape)

len(df["Lab_No"].unique())

In [ ]:
#check that it worked
display(df[df.duplicated([#'Lab_No', 
                  #'Test_Code', 
                  #'Test_Description',
                  'Date_Collected',
                  #'Time_collected',
                  #'UrNo', 
                  #'Surname', 
                  #'First_Names', 
                  #'Date_of_Birth',
                  #'Sex', 
                  #'Age', 
                  #'Category_Code', 
                  #'Facility_Code', 
                  #'Facility_Description',
                  #'Result', 
                  #'Result_Comment', 
                  #'DateTime_Collected',
                  #'Result_1',
                  'Patient_ID'], keep = False)].sort_values(['Patient_ID'],
                                                        ascending= True,
                                                        na_position='first'))



print("\n")

print("Unique Lab_No/tests: ", len(df.Lab_No.unique()))

print("\n")

print("Rows in df: ", len(df))

print("Patients df: ", len(df.Patient_ID.unique()))

In [ ]:
#save lab tests
df[['Patient_ID',
    'Lab_No', 
    'Date_Collected',
    'Time_collected',
    'Sex',
    'Age',
    "Result_1",
    'DateTime_Collected',
    'Area']].to_csv("Outputs/WrangledData/all_data_preCalcs.csv") 

**********************************

# <span style="color:magenta"> **STEP 6: CREATE FINAL DATAFRAME**<span>

Drop unneeded columns and save


In [ ]:
display(df.head())

df.columns

In [ ]:
#list(df.columns)

In [ ]:
df_final = df[["Patient_ID",
               "Age",
               'Sex',
               'Date_Collected',
               'Time_collected',
               'DateTime_Collected',
               'Result_1',
               'Facility_Description',
              'AgeGroup',
               'PostCode',
               'Area']]

print(df_final.dtypes)

df_final.head()

In [ ]:
#df_final["DateTime_Collected"] = pd.to_datetime(df_final["DateTime_Collected"], format='%Y-%m-%d %H:%M:%S', errors='coerce')
#df_final["Date_Collected"] = pd.to_datetime(df_final["Date_Collected"], format='%Y-%m-%d', errors='coerce')

#create column to add days since first test
df_final["DaySinceFirstTest"] = np.nan

#create column to add days since first POSITIVE test
df_final["DaySinceFirstPOSITIVETest"] = np.nan

#number of tests
df_final["NumberOfTests"] = np.nan

#number of tests since first positive
df_final["NumberOfTestsSinceFirstPOSITIVE"] = np.nan

#number of tests before first positive
df_final["NumberOfTestsBeforeFirstPOSITIVE"] = np.nan


#number of negative tests
df_final["NumberOfNegTests"] = np.nan

#number of positive tests
df_final["NumberOfPosTests"] = np.nan


# days since and including first poisitive test - this will be used to order visual
df_final["MaxDayTestSinceFirstPOSITIVETest"] = np.nan

# days test before first poisitive - this will be used to order visual
df_final["MaxDayTestBEFOREFirstPOSITIVETest"] = np.nan





#days to negative status - for filtering
df_final["DaysToNegStatus"] = np.nan

#days to negative status when first negative status is ignored - for filtering
df_final["DaysToNegStatus_sensitiv"] = np.nan


#days to biological negative status
df_final["DaysToNegStatus_biol"] = np.nan

#days to negative status when first negative status is ignored
df_final["DaysToNegStatus_sensitiv_biol"] = np.nan



#days to biological negative status OR final neg test
df_final["DaysToNegStatus_sensitivCensored"] = np.nan

#days to biological negative status OR final neg test when first negative status is ignored
df_final["DaysToNegStatus_sensitivUNCensored"] = np.nan




#number of result changes until first positive to DaysToNegStatus_biol
df_final["ResultChangeToNegStatus"] = np.nan

#number of result changes to DaysToNegStatus_sensitiv_biol
df_final["ResultChangeToFinalNegStatus"] = np.nan


# STATUS COLUMN
df_final["Status"]  = "NEG"


#column to indicate a status change from neg to pos and pos to neg
df_final["Status_Change"]  = np.nan

#Area for the first test
df_final["AreaFirstTest"]  = np.nan
df_final["PostCodeFirstTest"]  = np.nan
df_final["FacilityFirstTest"]  = np.nan



df_final.dtypes

In [ ]:
df_final.Patient_ID.value_counts()

In [ ]:
#Get list of patients
patient_list = df_final["Patient_ID"].unique()


for patient in patient_list:  
    
    
    df1 = df_final[df_final["Patient_ID"] == patient].sort_values(by = 'DateTime_Collected') #for each patient get their tests and sort according to datet_time
    
    
    ######################################### DAYS SINCE 'n' TEST

    #calcualte the date of first test
    first_date = df1["Date_Collected"].min()

    #get first instance of a POSITIVE result

    idx = (df1['Result_1'] >= "YES").idxmax()


    first_date_pos = df1.loc[idx, "Date_Collected"]



    #number of days since first test 
    df1["DaySinceFirstTest"] = (df1["Date_Collected"] - first_date)/ np.timedelta64(1, 'D')

    #number of days since first POSITIVE test    
    df1["DaySinceFirstPOSITIVETest"] = (df1["Date_Collected"] - first_date_pos)/ np.timedelta64(1, 'D')



    ######################################### DEFINE PATIENT STATUS

    #Define a new column for the previous day's results and copy results
    Result_Prev = df1["Result_1"]


    #Shift the results data down by one
    Result_Prev = Result_Prev.shift(periods=1)


    #Insert new results column into data frame
    df1.insert(7, 'Result_Prev', Result_Prev)





    #Define a new column for the previous day's result time and copy results
    DateTime_Collected_Prev = df1["DateTime_Collected"]


    #Shift the results data down by one
    DateTime_Collected_Prev = DateTime_Collected_Prev.shift(periods=1)


    #Insert new results column into data frame
    df1.insert(6, 'DateTime_Collected_Prev', DateTime_Collected_Prev)

    df1["TimeDifference"] = (df1["DateTime_Collected"] - df1['DateTime_Collected_Prev']).astype('timedelta64[h]')




    #Calculate status
    df1.loc[(df1['Result_Prev'].isnull()) & (df1['Result_1'] == "YES"), 'Status'] = "POS"

    df1.loc[(df1['Result_Prev'] == "YES") | (df1['Result_1'] == "YES"), 'Status'] = "POS"

    df1.loc[(df1['Result_Prev'] == "NO") & (df1['Result_1'] == "NO") & (df1["TimeDifference"] >= 24), 'Status'] = "NEG"





    ###################################### DEFINE NEGATIVE STATUS OCCURANCES


    #Define a new column for the previous day's results and copy results
    Status_Prev = df1["Status"]


    #Shift the results data down by one
    Status_Prev = Status_Prev.shift(periods=1)


    #Insert new results column into data frame
    df1.insert(16, 'Status_Prev', Status_Prev)


    #Calculate status change from POS to NEG
    df1.loc[(df1['Status_Prev'] == "POS") & (df1['Status'] == "NEG") & (df1['DaySinceFirstPOSITIVETest'] != 0), 'Status_Change'] = 1


    #Calculate status change from NEG to POS after initial positive
    df1.loc[(df1['Status_Prev'] == "NEG") & (df1['Status'] == "POS") & (df1['DaySinceFirstPOSITIVETest'] != 0), 'Status_Change'] = 2






    ######################################### REMOVE MULTIPLE NEGATIVE RESULTS AT END (REDUNDANT RESULTS)

    #get a list of row numbers the reverse the order
    rows = list(range(0, len(df1)))[::-1]

    #get the column numner for status
    columnStatus = df1.columns.get_loc("Status")

    #get the column numner for previous status
    columnStatusPrev = df1.columns.get_loc("Status_Prev")

    #loop through in reverse
    for i in rows:

        #if the last two status are NEG then remove the row
        if ((df1.iloc[i, columnStatus] == "NEG") & (df1.iloc[i, columnStatusPrev] == "NEG")):
            df1 = df1.iloc[:-1,]

        #otherwise get out of the loop
        else:
            break


    ###################################### DEFINE TIME TO FIRST NEGATIVE STATUS: BIOLOGICAL AND CLINICAL


    #Define a new column for the previous number of days since first positive and copy results
    DaysSinceFirstPos_Prev = df1["DaySinceFirstPOSITIVETest"]


    #Shift the results data down by one
    DaysSinceFirstPos_Prev = DaysSinceFirstPos_Prev.shift(periods=1)

    #Insert new results column into data frame
    df1.insert(16, 'DaysSinceFirstPos_Prev', DaysSinceFirstPos_Prev)

    #column number for days since first pos - previous test result for BIOLOGICALLY neg status 
    daysPrev = df1.columns.get_loc("DaysSinceFirstPos_Prev")



    #if a negative status is achieved at some point
    if (len(df1[df1["Status_Change"] == 1]) > 0) :

        #BIOLOGICAL time to negative is the FIRST occurance of the change to negative status
        df1["DaysToNegStatus_biol"] = df1[df1["Status_Change"] == 1].iloc[0,daysPrev]

        #BIOLOGICAL time to negative is the LAST data row
        df1["DaysToNegStatus_sensitiv_biol"] = df1["DaySinceFirstPOSITIVETest"].max() #if last status is pos

        #print(df1["DaySinceFirstPOSITIVETest"].max())

        #display(df1["DaySinceFirstPOSITIVETest"])

        #CLINICAL time to negative is the FIRST occurance of the change to negative status
        df1["DaysToNegStatus"] = df1.loc[(df1['Status_Change'] == 1.0).idxmax(), "DaySinceFirstPOSITIVETest"]

        #CLINICAL time to negative is the LAST data row
        df1["DaysToNegStatus_sensitiv"] = df1["DaySinceFirstPOSITIVETest"].max()



    if (len(df1[df1["Status_Change"] == 1]) > 0 ) & (df1.iloc[-1,df1.columns.get_loc("Status")]  == "NEG"):

        df1["DaysToNegStatus_sensitiv_biol"] = df1.iloc[len(df1)-1,df1.columns.get_loc("DaysSinceFirstPos_Prev")]  #if the last status is neg






    #if a negative status is never achieved in a previously positive patient   
    if (len(df1[df1["Status_Change"] == 1]) == 0) & (df1[df1["DaySinceFirstPOSITIVETest"] == 0]["Status"].item() == "POS"):

        df1["DaysToNegStatus_biol"] = df1["DaySinceFirstPOSITIVETest"].max()

        df1["DaysToNegStatus_sensitiv_biol"] = df1["DaySinceFirstPOSITIVETest"].max()

        df1["DaysToNegStatus"] = df1["DaySinceFirstPOSITIVETest"].max()

        df1["DaysToNegStatus_sensitiv"] = df1["DaySinceFirstPOSITIVETest"].max()





    ###################################### DEFINE TIME TO FIRST NEGATIVE STATUS: BIOLOGICAL OR FINAL NEG TEST   

     #check to see that days to neg is not null
    if pd.notnull(df1.DaysToNegStatus.iloc[0]):

        #days to biological negative status OR final neg test
        days = df1.DaysToNegStatus.iloc[0]

        days_negAch = df1.DaysToNegStatus_biol.iloc[0]

        #if negative status - then use days to neg status
        if (df1[(df1.DaySinceFirstPOSITIVETest == days)].Status == "NEG").item():
            df1["DaysToNegStatus_sensitivCensored"] = days_negAch

        #if postivive status BUT last test neg
        if (((df1[(df1.DaySinceFirstPOSITIVETest == days)].Status == "POS") & (df1[(df1.DaySinceFirstPOSITIVETest == days)].Result_1 == "NO"))).item():

            df1["DaysToNegStatus_sensitivCensored"] = days



        #days to biological negative status OR final neg test when first negative status is ignored
        days = df1.DaysToNegStatus_sensitiv.iloc[0]

        days_negAch = df1.DaysToNegStatus_sensitiv_biol.iloc[0]

        #if negative status - then use days to neg status
        if (df1[(df1.DaySinceFirstPOSITIVETest == days)].Status == "NEG").item():
            df1["DaysToNegStatus_sensitivUNCensored"] = days_negAch

        #if postivive status BUT last test neg
        if (((df1[(df1.DaySinceFirstPOSITIVETest == days)].Status == "POS") & (df1[(df1.DaySinceFirstPOSITIVETest == days)].Result_1 == "NO"))).item():

            df1["DaysToNegStatus_sensitivUNCensored"] = days




    ######################################### MAX and MIN DAYS SINCE 'n' TEST


    #MAX days after first positive test
    df1["MaxDayTestSinceFirstPOSITIVETest"] = df1["DaySinceFirstPOSITIVETest"].max()


    #MAX days before first positive test
    df1["MaxDayTestBEFOREFirstPOSITIVETest"] = df1["DaySinceFirstPOSITIVETest"].min()



    #number of tests done after and including first positive
    df1["NumberOfTestsSinceFirstPOSITIVE"] = len(df1[(df1["DaySinceFirstPOSITIVETest"] >=0)])

    #number of tests done befre and excluding first positive
    df1["NumberOfTestsBeforeFirstPOSITIVE"] = len(df1[df1["DaySinceFirstPOSITIVETest"] < 0])




    #################################### COUNT THE NUMBER OF CHANGES IN RESULTS FROM POS TO NEG



    #number of result changes until first positive
    df1["ResultChangeToNegStatus"] = len(df1[(df1["Result_1"] == "YES") & (df1["Result_Prev"] == "NO") & (df1["DaySinceFirstPOSITIVETest"] <=df1['DaysToNegStatus'])])

    #number of result changes until final positive
    df1["ResultChangeToFinalNegStatus"] = len(df1[(df1["Result_1"] == "YES") & (df1["Result_Prev"] == "NO")])





    ######################################### NUMBER OF TESTS

    #TOTAL number of tests
    df1["NumberOfTests"] = len(df1)


    # number of negative tests  
    df1["NumberOfNegTests"] = len(df1[(df1["Result_1"] == "NO") & (df1["DaySinceFirstPOSITIVETest"] >=0)])

    #number of positive tests

    df1["NumberOfPosTests"] = len(df1[(df1["Result_1"] == "YES") & (df1["DaySinceFirstPOSITIVETest"] >= 0)])



    ########################### AREA OF FIRST TEST 

    df1["AreaFirstTest"]  =  df1.iloc[0, df1.columns.get_loc("Area")]
    df1["PostCodeFirstTest"]  = df1.iloc[0, df1.columns.get_loc('PostCode')]
    df1["FacilityFirstTest"]  = df1.iloc[0, df1.columns.get_loc('Facility_Description')]
    
    
    
    
    
    df_final.update(df1)
        

In [ ]:
df_final1 = df_final.copy()

display(df_final1)

df_final1.dtypes

In [ ]:
#drop all the values 
df_final1 = df_final1[df_final1['DaySinceFirstTest'].notna()]

In [ ]:
df_final1["NumberOfTestsSinceFirstPOSITIVE"].max()

In [ ]:
df_final1["DaySinceFirstTest"] = df_final1["DaySinceFirstTest"].astype(int)
df_final1["DaySinceFirstPOSITIVETest"] = df_final1["DaySinceFirstPOSITIVETest"].astype(int)
df_final1["MaxDayTestSinceFirstPOSITIVETest"] = df_final1["MaxDayTestSinceFirstPOSITIVETest"].astype(int)
df_final1["Patient_ID"] = df_final1["Patient_ID"].astype(int)
df_final1["Age"] = df_final1["Age"].astype(int)




df_final1

In [ ]:
#check that there are no groups with < 5 patients
df_final1.AgeGroup.value_counts()

# SAVE FILES

In [ ]:
#save file
df_final1.to_csv("Outputs/WrangledData/all_results.csv") 
